In [75]:
### Library Imports
import pandas as pd
import numpy as np
### Script Imports
import utils


In [13]:
### Clean Accounts

def clean_account():
    
    ### Import the loans file
    na_values= ['None', '?',]
    df = pd.read_csv('../../project/banking_data/account.csv', sep=';',na_values=na_values)
    
    ### Date conversion
    return utils.date_conversion(df,'date')

In [24]:
def clean_card(test = False):
    ### Import the loans file
    na_values= ['None', '?',]
    if(test):
        df = pd.read_csv('../../project/banking_data/card_test.csv', sep=';',na_values=na_values)
    else:
        df = pd.read_csv('../../project/banking_data/card_train.csv', sep=';',na_values=na_values)

    return utils.date_conversion(df,'issued')


In [15]:
def clean_client():
    ### Import the clients file
    na_values= ['None', '?',]
    df = pd.read_csv('../../project/banking_data/client.csv', sep=';',na_values=na_values)
    

    ### Date conversion with creation of gender column
    df = utils.date_conversion_genders(df,'birth_number')
    
    ### Reordering Columns
    cols = df.columns.tolist()
    cols = cols[0:2] + cols[3:4] + cols[2:3]
    return df[cols]

In [16]:
def clean_district():
    ### Import the loans file
    na_values= ['None', '?',]
    df = pd.read_csv('../../project/banking_data/district.csv', sep=';',na_values=na_values)

    ### Replace null values in the '95 column for the ones on the '96 column
    districts = df.copy()
    districts["no. of commited crimes \'95 "] = districts["no. of commited crimes \'95 "].combine_first(districts["no. of commited crimes \'96 "])
    districts["unemploymant rate \'95 "] = districts["unemploymant rate \'95 "].combine_first(districts["unemploymant rate \'96 "])
    
    ### Added columns for crime analysis
    districts['crime_growth'] = districts["no. of commited crimes '96 "] - districts["no. of commited crimes '95 "]
    districts['total_crime'] = districts["no. of commited crimes '96 "] + districts["no. of commited crimes '95 "]

    ### Added column for uneployment growth analysis
    districts['unemploymant_growth'] = districts["unemploymant rate '96 "] - districts["unemploymant rate '95 "]

    ### Reorder columns
    cols = districts.columns.tolist()
    cols = cols[:13] + cols[-1:] + cols[13:-1]

    return districts[cols]    
    

In [17]:
def clean_loans(test=False):
    ### Import the loans file
    na_values= ['None', '?',]
    if(test):
        df = pd.read_csv('../../project/banking_data/loan_test.csv', sep=';',na_values=na_values)
    else:
        df = pd.read_csv('../../project/banking_data/loan_train.csv', sep=';',na_values=na_values)
    
    return utils.date_conversion(df,'date')

In [18]:
def clean_trans(test = False):
    ### Import the trans file
    
    na_values= ['None', '?',]
    
    if test:
        df = pd.read_csv('../../project/banking_data/trans_test.csv', sep=';',na_values=na_values)
    else:
        df = pd.read_csv('../../project/banking_data/trans_train.csv', sep=';',na_values=na_values)
    


    ### Convert date format
    trans = utils.date_conversion(df,'date')

    ### Drop columns with too many missing values
    trans = utils.drop_null_columns(trans,0.4)

    ### Replace null values from the operation column for the most common occurence
    trans['operation'] = trans['operation'].fillna(trans['operation'].value_counts().idxmax())
    
    return trans

In [27]:
### Merge Data


# def get_final_df(test=False):
test= False

accounts = clean_account()
cards = clean_card(test)
clients = clean_client()
districts = clean_district()
loans = clean_loans(test)
trans = clean_trans(test)
disp = pd.read_csv('../../project/banking_data/disp.csv', sep=';')



C:\Users\utilizador\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2901: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [83]:
### TODO:DO a New Merge with everything
# ### Merge everything with sequel queries

cards
disp

# accounts.merge(disp, left_on='lkey', right_on='rkey')
cardClients = cards.rename(columns={"type":"card_type", "issued":"card_date"})
cardClients = cards.merge(disp, how='left', on='disp_id')
cardClients = cardClients[cardClients["type"] == "OWNER"]
cardClients


clientDistricts = clients.merge(districts, left_on='district_id', right_on='code ')
clientDistricts.drop(columns=['code '])
clientDistricts


accountDistricts = accounts.merge(districts, left_on='district_id', right_on='code ')
accountDistricts.drop(columns=['code '])
accountDistricts

cardWithClient = cards.merge(disp).drop(columns=["card_id","disp_id","client_id","type"])
loanClientsWithNoCard = [i for i in loans["account_id"] if not i in list(cardWithClient["account_id"])]
cardClientsWithNoLoan = [i for i in cardWithClient["account_id"] if not i in list(loans["account_id"])]
cardWithClient = cardWithClient.drop(index=cardWithClient[cardWithClient["account_id"].isin(cardClientsWithNoLoan)].index)
loanClientsWithNoCard = np.array([loanClientsWithNoCard,["no card"]*len(loanClientsWithNoCard),[0]*len(loanClientsWithNoCard)]).transpose()
loanClientsWithNoCard = pd.DataFrame(loanClientsWithNoCard,columns=["account_id","card_type", "card issue"])
cardWithClient = cardWithClient.append(loanClientsWithNoCard)
cardWithClient["account_id"] = cardWithClient["account_id"].astype(int)

# accountClient = disp.merge(accountDistricts)
# accountClient = accountClient.merge(clientDistricts)

accountClient = disp.merge(accountDistricts)
accountClient = accountClient.merge(clientDistricts)
accountClient = accountClient.drop(columns="client_id")
accountClient = accountClient[accountClient["type"] == "OWNER"]


loanFinal = loans.merge(accountClient, on="account_id")
loanFinal = loanFinal.merge(cardWithClient, on="account_id")
loanFinal = loanFinal.merge(trans, on="account_id")
loanFinal = loanFinal.drop(columns=["account_id", "disp_id", "type_x"])

loanFinal



C:\Users\utilizador\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,loan_id,date_x,amount_x,duration,payments,status,district_id,frequency,date_y,code,...,gender,card issue,card_date,card_type,trans_id,date,type_y,operation,amount_y,balance
0,5314,05-07-1993,96396,12,8033,-1,30,weekly issuance,22-03-1993,30,...,female,0,NaN,no card,523621,22-03-1993,credit,credit in cash,1100.0,1100.0
1,5314,05-07-1993,96396,12,8033,-1,30,weekly issuance,22-03-1993,30,...,female,0,NaN,no card,524054,21-04-1993,credit,credit in cash,9900.0,11000.0
2,5314,05-07-1993,96396,12,8033,-1,30,weekly issuance,22-03-1993,30,...,female,0,NaN,no card,524055,21-05-1993,credit,credit in cash,5800.0,16800.0
3,5314,05-07-1993,96396,12,8033,-1,30,weekly issuance,22-03-1993,30,...,female,0,NaN,no card,524056,20-06-1993,credit,credit in cash,3300.0,20100.0
4,5316,11-07-1993,165960,36,4610,1,46,monthly issuance,13-02-1993,46,...,male,0,NaN,no card,527445,13-02-1993,credit,credit in cash,700.0,700.0
5,5316,11-07-1993,165960,36,4610,1,46,monthly issuance,13-02-1993,46,...,male,0,NaN,no card,3589600,28-02-1993,credit,withdrawal in cash,2.9,702.9
6,5316,11-07-1993,165960,36,4610,1,46,monthly issuance,13-02-1993,46,...,male,0,NaN,no card,527456,28-02-1993,credit,credit in cash,18972.0,19674.9
7,5316,11-07-1993,165960,36,4610,1,46,monthly issuance,13-02-1993,46,...,male,0,NaN,no card,528025,15-03-1993,credit,credit in cash,3700.0,23374.9
8,5316,11-07-1993,165960,36,4610,1,46,monthly issuance,13-02-1993,46,...,male,0,NaN,no card,527446,16-03-1993,credit,credit in cash,8368.0,31742.9
9,5316,11-07-1993,165960,36,4610,1,46,monthly issuance,13-02-1993,46,...,male,0,NaN,no card,527466,20-03-1993,credit,credit in cash,36574.0,68316.9
